# Interactive Time Series Visualization Assignment

In [1]:
import pandas as pd
import plotly_express as px
from ipywidgets import interact

### Load the Online Retail.xlsx Excel file into a Pandas dataframe.

In [2]:
url = 'https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%202/Online%20Retail.xlsx'

df = pd.read_excel(url)
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396034 entries, 0 to 396033
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396034 non-null  int64         
 1   InvoiceDate  396034 non-null  datetime64[ns]
 2   StockCode    396034 non-null  object        
 3   Description  396034 non-null  object        
 4   Quantity     396034 non-null  int64         
 5   UnitPrice    396034 non-null  float64       
 6   Revenue      396034 non-null  float64       
 7   CustomerID   396034 non-null  int64         
 8   Country      396034 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 27.2+ MB


### Create an interactive histogram to explore the distributions of daily (by date) Quantity, UnitPrice, and Revenue. Include a dropdown widget where you can switch between the three fields.

**Hint:** You will need to create a Date field and then aggregate on it. The Quantity and Revenue fields should be summed and the UnitPrice field should be averaged (mean).

In [14]:
#daily_units = feb.groupby('Date', as_index=False).agg({'Quantity': 'sum'})

#daily_units = products.pivot_table(index='Date', columns='ProductID', values='Quantity', aggfunc='sum')

In [15]:
def ihistogram(df, field, title=''):
    fig = px.histogram(df, x=field, title=title,
                      template='none')
    
    fig.update_yaxes(title_text='Number of Records')
    fig.update_traces(marker_color='lightskyblue',
                     marker_line_color='black',
                     marker_line_width=1)
    
    fig.show()

In [13]:
df['Date'] = df['InvoiceDate'].dt.date
part1 = df.pivot_table(index='Date', values=['Quantity', 'Revenue'], aggfunc='sum')
part2 = df.pivot_table(index='Date', values='UnitPrice', aggfunc='mean')
dailies = pd.concat([part1, part2], axis=1)

In [18]:
@interact(Measure=['Quantity', 'UnitPrice', 'Revenue'])

def viz(Measure):
    ihistogram(dailies, Measure, title='Measures of Sales by Day')

interactive(children=(Dropdown(description='Measure', options=('Quantity', 'UnitPrice', 'Revenue'), value='Qua…

### Create an interactive line chart where you can explore Quantity and Revenue by day (date) for each country. Include two dropdown widgets - one to choose between the Quantity and Revenue fields and the other to choose the Country.

In [19]:
def ilinechart(df, x, y, groups=None, title=''):
    fig = px.line(df, x=x, y=y, color=groups, title=title, 
                  template='none')#.update(layout=dict(title=dict(x=0.5)))

#     This section is for when there are multiple lines in the chart.
#     for item in range(len(fig.data)):
#         fig.data[item].update(name=fig.data[item]['name'].split('=')[1])

    fig.show()

In [20]:
df['Country'].unique()

array(['United Kingdom', 'Spain', 'Cyprus', 'EIRE', 'Portugal',
       'Netherlands', 'Australia', 'Singapore', 'Switzerland', 'Finland',
       'Channel Islands', 'France', 'Malta', 'Italy', 'Israel', 'Germany',
       'Austria', 'Norway', 'Denmark', 'Sweden', 'Belgium', 'Canada',
       'Poland', 'Japan', 'Iceland', 'Greece', 'Lebanon', 'South Africa',
       'Czech Republic', 'USA', 'Brazil', 'United Arab Emirates',
       'Lithuania', 'Saudi Arabia', 'Bahrain'], dtype=object)

In [26]:
@interact(Measure=['Quantity', 'Revenue'],
         Country=df['Country'].unique())


def viz(Measure, Country):
    filtered = df[df['Country'] == Country]
    aggregated = filtered.groupby('Date').agg({'Quantity': 'sum', 'Revenue': 'sum'}).reset_index()
    ilinechart(aggregated, 'Date', Measure, title=f'Total Daily Sales {Measure} for {Country}')

interactive(children=(Dropdown(description='Measure', options=('Quantity', 'Revenue'), value='Quantity'), Drop…

### Create an interactive scatter plot showing the relationships between daily Quantity, UnitPrice, and Revenue for the United Kingdom. Include two dropdown boxes that let you choose between the 3 fields - one for the x axis and one for the y axis of your scatter plot. Size the data points according to Revenue.

In [29]:
def iscatter(df, x, y, color=None, size=None, title=''):
    fig = px.scatter(df, x=x, y=y, color=color, size=size, title=title, template='none')
    
    #for item in range(len(fig.data)):
        #fig.data[item].update(name=fig.data[item]['name'].split('=')[1])
        
    fig.update_traces(marker_line_color='black',
                     marker_line_width=1)
    
    fig.show()

In [31]:
@interact(X=['Quantity', 'Revenue', 'UnitPrice'],
         Y=['Quantity', 'Revenue', 'UnitPrice'])


def viz(X, Y):
    filtered = df[df['Country'] == 'United Kingdom']
    aggregated = filtered.groupby('Date').agg({'Quantity': 'sum', 'Revenue': 'sum', 'UnitPrice': 'mean'}).reset_index()
    iscatter(aggregated, X, Y, size=aggregated['Revenue'], title=f'{Y} vs {X} for the United Kingdom')

interactive(children=(Dropdown(description='X', options=('Quantity', 'Revenue', 'UnitPrice'), value='Quantity'…

### Create a bar chart showing the top X products sold (by quantity) in the United Kingdom in a specific month. Use a dropdown box to select the month name and a slider to show the top X products. The range for X should be from 5 to 25. Make sure the bars are sorted in descending order according to their total quantity sold for the month.

Bonus points if you can figure out how to ensure that the month names in the dropdown appear in their correct order (January, February, March, April, etc.).

In [37]:
df['Month'] = df['InvoiceDate'].dt.month
df['Month'].unique()

array([12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [32]:
def ibarchart(df, x, y, order=None, title=''):
    fig = px.bar(df, x=x, y=y, title=title, template='none')
    
    fig.update_traces(marker_color='lightskyblue',
                     marker_line_color='black',
                     marker_line_width=1)
    
    fig.update_layout(xaxis={'categoryorder': 'array',
                            'categoryarray': order})
    
    fig.show()

In [52]:
print(sorted(df['Month'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [56]:
@interact(X=(5,25),
         Month=sorted(df['Month'].unique()))


#daily_units = products.pivot_table(index='Date', columns='ProductID', values='Quantity', aggfunc='sum')

def viz(X, Month):
    filtered = df[(df['Country'] == 'United Kingdom') & (df['Month'] == Month)]
    aggregated = filtered.pivot_table(index='Description', values='Quantity', aggfunc='sum').reset_index().sort_values(by='Quantity', ascending=False).head(X)
    order = sorted(df['Month'].unique())
    ibarchart(aggregated, 'Description', 'Quantity', order=order, title=f'Top {X} Products in the United Kingdom in Month #{Month}')

interactive(children=(IntSlider(value=15, description='X', max=25, min=5), Dropdown(description='Month', optio…